In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
missing_query = f"""
SELECT
    * 
FROM
    (
    SELECT
        repo_info.repo_id,
        repo.repo_name,
repo.repo_git, 
        MAX ( pull_request_count ) AS max_pr_count,
        COUNT ( * ) AS meta_count 
    FROM
        repo_info,
        repo -- WHERE issues_enabled = 'true' 
    WHERE
        pull_request_count >= 1
        AND repo.repo_id = repo_info.repo_id 
    GROUP BY
        repo_info.repo_id,
        repo.repo_name, 
 repo.repo_git 
    ORDER BY
        repo_info.repo_id,
        repo.repo_name 
    ) yy
    LEFT OUTER JOIN (
    SELECT A
        .repo_id,
        A.repo_name,
        b.pull_request_count,
        d.repo_id AS pull_request_repo_id,
        e.last_collected,
        f.last_pr_collected,
        COUNT ( * ) AS pull_requests_collected,
        ( b.pull_request_count - COUNT ( * ) ) AS pull_requests_missing,
        ABS ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_abs,
        ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_prs 
    FROM
        augur_data.repo A,
        augur_data.pull_requests d,
        augur_data.repo_info b,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_collected FROM augur_data.repo_info GROUP BY repo_id ORDER BY repo_id ) e,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_pr_collected FROM augur_data.pull_requests GROUP BY repo_id ORDER BY repo_id ) f 
    WHERE
        A.repo_id = b.repo_id 
        AND LOWER ( A.repo_git ) LIKE'%github.com%' 
        AND A.repo_id = d.repo_id 
        AND b.repo_id = d.repo_id 
        AND e.repo_id = A.repo_id 
        AND b.data_collection_date = e.last_collected 
        AND f.repo_id = A.repo_id -- AND d.pull_request_id IS NULL
        
    GROUP BY
        A.repo_id,
        d.repo_id,
        b.pull_request_count,
        e.last_collected,
        f.last_pr_collected 
    ORDER BY
        ratio_abs desc
    ) zz ON yy.repo_id = zz.repo_id 
ORDER BY
    ratio_abs;
    """
missing_data = pd.read_sql_query(missing_query, con=engine)
#print(missing_data)

In [3]:
df = missing_data.loc[missing_data['ratio_abs'] < .95]
#print(df)
df1 = df[['repo_id', 'repo_git', 'last_pr_collected', 'pull_requests_missing']]
print(df1)
df1.to_csv('/Users/dafoster/gitrepos/augur-pivotal/code/health_scripts/output/missing.csv')

    repo_id  repo_id                                           repo_git  \
0     26005  26005.0            https://github.com/SteeltoeOSS/MainSite   
1     30420  30420.0  https://github.com/vmware-samples/secure-state...   
2     30271  30271.0  https://github.com/carbonblack/cb-threatconnec...   
3     27511  27511.0             https://github.com/pivotal-cf/docs-pas   
4     26009  26009.0    https://github.com/Pivotal-DataFabric/qp-images   
5     30233  30233.0  https://github.com/bitnami/bitnami-docker-mong...   
6     30434  30434.0  https://github.com/vmware-samples/rabbitmq-oau...   
7     26439  26439.0      https://github.com/cf-platform-eng/ksm-sample   
8     30261  30261.0  https://github.com/carbonblack/cb-taxii-connector   
9     30191  30191.0    https://github.com/bitnami/bitnami-docker-spark   
10    30182  30182.0  https://github.com/bitnami/bitnami-docker-airf...   
11    27650  27650.0    https://github.com/pivotal-cf/docs-book-windows   
12    30151  30151.0     